In [2]:
#imports
#from fastai.imports import *
#from fastai.structured import *

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from IPython.display import display
from sklearn import metrics
#from pandas_summary import DataFrameSummary 

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBRegressor as xgb


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_raw = pd.read_csv('train.csv', low_memory=False)
df_raw

,Product_Identifier,Supermarket_Identifier,Product_Supermarket_Identifier,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type,Product_Supermarket_Sales
0,DRA12,CHUKWUDI010,DRA12_CHUKWUDI010,11.600,Low Fat,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
1,DRA12,CHUKWUDI013,DRA12_CHUKWUDI013,11.600,Low Fat,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
2,DRA12,CHUKWUDI017,DRA12_CHUKWUDI017,11.600,Low Fat,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
3,DRA12,CHUKWUDI018,DRA12_CHUKWUDI018,11.600,Low Fat,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
4,DRA12,CHUKWUDI035,DRA12_CHUKWUDI035,11.600,Ultra Low fat,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77
5,DRA12,CHUKWUDI045,DRA12_CHUKWUDI045,11.600,Low Fat,0.000000,Soft Drinks,354.04,2009,NaN,Cluster 2,Supermarket Type1,9572.54
6,DRA24,CHUKWUDI010,DRA24_CHUKWUDI010,19.350,Normal Fat,0.066832,Soft Drinks,409.72,2005,NaN,Cluster 3,Grocery Store,818.93
7,DRA24,CHUKWUDI013,DRA24_CHUKWUDI013,19.350,Normal Fat,0.039895,Soft Drinks,406.22,1994,High,Cluster 3,Supermarket Type1,11055.61
8,DRA24,CHUKWUDI017,DRA24_CHUKWUDI017,19.350,Normal Fat,0.040154,Soft Drinks,411.72,2014,NaN,Cluster 2,Supermarket Type1,2866.27
9,DRA24,CHUKWUDI019,DRA24_CHUKWUDI019,NaN,Normal Fat,0.069909,Soft Drinks,408.22,1992,Small,Cluster 1,Grocery Store,1228.40


In [10]:
df_raw.Supermarket_Identifier.value_counts()

CHUKWUDI049    620
CHUKWUDI013    594
CHUKWUDI017    583
CHUKWUDI035    515
CHUKWUDI046    509
CHUKWUDI018    500
CHUKWUDI045    483
CHUKWUDI027    462
CHUKWUDI010    384
CHUKWUDI019    340
Name: Supermarket_Identifier, dtype: int64

In [25]:
df_test = pd.read_csv('test.csv')
#add_datepart(df_test, 'Supermarket_Opening_Year')
df_test.head()

,Product_Identifier,Supermarket_Identifier,Product_Supermarket_Identifier,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type
0,DRA59,CHUKWUDI010,DRA59_CHUKWUDI010,8.270,Normal Fat,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store
1,DRA59,CHUKWUDI013,DRA59_CHUKWUDI013,8.270,Normal Fat,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1
2,DRB01,CHUKWUDI013,DRB01_CHUKWUDI013,7.390,Low Fat,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1
3,DRB13,CHUKWUDI010,DRB13_CHUKWUDI010,6.115,Normal Fat,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store
4,DRB13,CHUKWUDI013,DRB13_CHUKWUDI013,6.115,Normal Fat,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1


In [14]:
# Convert string data types into categorical, from fastai library
train_cats(df_raw)

train_cats(df_test)

In [ ]:
# convert all objects into numeric, specially from fastai library
df, y, nas = proc_df(df_r, 'Product_Supermarket_Sales')

In [5]:
# The product_supermarket_sales column has a consistent approx. factor of the product prices for each supermarket type
df_raw['leakage'] = df_raw.Product_Supermarket_Sales/df_raw.Product_Price
df_raw

,Product_Identifier,Supermarket_Identifier,Product_Supermarket_Identifier,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type,Product_Supermarket_Sales,leakage
0,DRA12,CHUKWUDI010,DRA12_CHUKWUDI010,11.600,Low Fat,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08,1.983219
1,DRA12,CHUKWUDI013,DRA12_CHUKWUDI013,11.600,Low Fat,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69,17.936676
2,DRA12,CHUKWUDI017,DRA12_CHUKWUDI017,11.600,Low Fat,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69,18.192337
3,DRA12,CHUKWUDI018,DRA12_CHUKWUDI018,11.600,Low Fat,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23,5.991522
4,DRA12,CHUKWUDI035,DRA12_CHUKWUDI035,11.600,Ultra Low fat,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77,6.995039
5,DRA12,CHUKWUDI045,DRA12_CHUKWUDI045,11.600,Low Fat,0.000000,Soft Drinks,354.04,2009,NaN,Cluster 2,Supermarket Type1,9572.54,27.038018
6,DRA24,CHUKWUDI010,DRA24_CHUKWUDI010,19.350,Normal Fat,0.066832,Soft Drinks,409.72,2005,NaN,Cluster 3,Grocery Store,818.93,1.998755
7,DRA24,CHUKWUDI013,DRA24_CHUKWUDI013,19.350,Normal Fat,0.039895,Soft Drinks,406.22,1994,High,Cluster 3,Supermarket Type1,11055.61,27.215819
8,DRA24,CHUKWUDI017,DRA24_CHUKWUDI017,19.350,Normal Fat,0.040154,Soft Drinks,411.72,2014,NaN,Cluster 2,Supermarket Type1,2866.27,6.961697
9,DRA24,CHUKWUDI019,DRA24_CHUKWUDI019,NaN,Normal Fat,0.069909,Soft Drinks,408.22,1992,Small,Cluster 1,Grocery Store,1228.40,3.009162


In [11]:
# The mean is the best measure of that factor
df_raw.groupby('Supermarket_Identifier')['leakage'].mean()

Supermarket_Identifier
CHUKWUDI010     2.316119
CHUKWUDI013    15.669220
CHUKWUDI017    16.838994
CHUKWUDI018    14.273898
CHUKWUDI019     2.400813
CHUKWUDI027    26.452247
CHUKWUDI035    17.074440
CHUKWUDI045    15.755592
CHUKWUDI046    16.311350
CHUKWUDI049    16.815348
Name: leakage, dtype: float64

In [11]:
df_raw.groupby('Supermarket_Type')['leakage'].mean()

Supermarket_Type
Grocery Store         2.355893
Supermarket Type1    16.421287
Supermarket Type2    14.273898
Supermarket Type3    26.452247
Name: leakage, dtype: float64

In [12]:
df_raw['leakage'].mean()

15.094085021706396

In [ ]:
feats = []
for feat in df_test.columns:
    feats.append(feat)
feats

In [16]:
feats.remove('Product_Supermarket_Sales')

In [8]:
for f in feats:
    df_test = df_test.drop([f], axis=1)

In [19]:
df_raw.head()

,Product_Supermarket_Identifier,Product_Price,Supermarket_Type
0,DRA12_CHUKWUDI010,357.54,Grocery Store
1,DRA12_CHUKWUDI013,355.79,Supermarket Type1
2,DRA12_CHUKWUDI017,350.79,Supermarket Type1
3,DRA12_CHUKWUDI018,355.04,Supermarket Type2
4,DRA12_CHUKWUDI035,354.79,Supermarket Type1


In [9]:
# Define a new column to insert all the predictions ... still ones for now
df_test['Pred'] = np.ones((len(df_test)))

In [25]:
leak = {'Grocery Store': 2.355893,
        'Supermarket Type1':16.421287,
        'Supermarket Type2':14.273898,
        'Supermarket Type3':26.452247}

In [ ]:
CHUKWUDI010     2.316119
CHUKWUDI013    15.669220
CHUKWUDI017    16.838994
CHUKWUDI018    14.273898
CHUKWUDI019     2.400813
CHUKWUDI027    26.452247
CHUKWUDI035    17.074440
CHUKWUDI045    15.755592
CHUKWUDI046    16.311350
CHUKWUDI049    16.815348

In [34]:
def multiply(column1, column0):
    if column1=='CHUKWUDI010':
        return column0*2.316119
    elif column1=='CHUKWUDI013':
        return  column0*15.669220
    elif  column1== 'CHUKWUDI017':
        return  column0*16.838994
    elif column1== 'CHUKWUDI018':
        return  column0*14.273898
    elif column1 == 'CHUKWUDI019':
        return  column0*2.400813
    elif column1 == 'CHUKWUDI027':
        return  column0*26.452247
    elif column1 == 'CHUKWUDI035':
        return  column0*17.074440
    elif column1== 'CHUKWUDI045':
        return  column0*15.755592
    elif column1 == 'CHUKWUDI046':
        return  column0*16.311350
    elif column1 == 'CHUKWUDI049':
        return  column0*16.815348
    

In [36]:
df_test['Product_Supermarket_Sales']=df_test.apply(lambda x: multiply(x['Supermarket_Identifier'],x['Product_Price']),axis=1)
df_test.head()

,Product_Identifier,Supermarket_Identifier,Product_Supermarket_Identifier,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type,Product_Supermarket_Sales
0,DRA59,CHUKWUDI010,DRA59_CHUKWUDI010,8.270,Normal Fat,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store,1065.368418
1,DRA59,CHUKWUDI013,DRA59_CHUKWUDI013,8.270,Normal Fat,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1,7285.873916
2,DRB01,CHUKWUDI013,DRB01_CHUKWUDI013,7.390,Low Fat,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1,7480.172244
3,DRB13,CHUKWUDI010,DRB13_CHUKWUDI010,6.115,Normal Fat,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store,1094.667323
4,DRB13,CHUKWUDI013,DRB13_CHUKWUDI013,6.115,Normal Fat,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1,7413.578059


In [ ]:
# Use this factor to multiply each row in the test data set
for e in range(len(df_test)):
    if df_test.Supermarket_Identifier[e] == 'CHUKWUDI010':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*2.316119
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI013':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*15.669220
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI017':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*16.838994
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI018':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*14.273898
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI019':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*2.400813
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI027':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*26.452247
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI035':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*17.074440
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI045':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*15.755592
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI046':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*16.311350
    elif df_test.Supermarket_Identifier[e] == 'CHUKWUDI049':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*16.815348
df_test

In [ ]:
# Use this factor to multiply each row in the test data set
for e in range(len(df_test)):
    if df_test.Supermarket_Type[e] == 'Grocery Store':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*2.355893
    elif df_test.Supermarket_Type[e] == 'Supermarket Type1':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*16.421287
    elif df_test.Supermarket_Type[e] == 'Supermarket Type2':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*14.273898
    elif df_test.Supermarket_Type[e] == 'Supermarket Type3':
        df_test['Product_Price'][e] = df_test.Product_Price[e]*26.452247
df_test

In [30]:
def rmse(x, y): return math.sqrt(((x - y)**2).mean())

In [15]:
df_test.Product_Supermarket_Identifier

0       DRA59_CHUKWUDI010
1       DRA59_CHUKWUDI013
2       DRB01_CHUKWUDI013
3       DRB13_CHUKWUDI010
4       DRB13_CHUKWUDI013
5       DRB25_CHUKWUDI017
6       DRB25_CHUKWUDI027
7       DRB25_CHUKWUDI035
8       DRB48_CHUKWUDI017
9       DRB48_CHUKWUDI027
10      DRB48_CHUKWUDI035
11      DRB48_CHUKWUDI046
12      DRB48_CHUKWUDI049
13      DRC01_CHUKWUDI017
14      DRC01_CHUKWUDI018
15      DRC01_CHUKWUDI027
16      DRC01_CHUKWUDI045
17      DRC01_CHUKWUDI046
18      DRC13_CHUKWUDI035
19      DRC13_CHUKWUDI045
20      DRC25_CHUKWUDI018
21      DRC25_CHUKWUDI019
22      DRC25_CHUKWUDI027
23      DRC25_CHUKWUDI045
24      DRC25_CHUKWUDI046
25      DRC27_CHUKWUDI010
26      DRC27_CHUKWUDI018
27      DRC27_CHUKWUDI049
28      DRC36_CHUKWUDI013
29      DRD01_CHUKWUDI017
              ...        
3502    NCY41_CHUKWUDI035
3503    NCY41_CHUKWUDI045
3504    NCY41_CHUKWUDI046
3505    NCY41_CHUKWUDI049
3506    NCY53_CHUKWUDI017
3507    NCY53_CHUKWUDI027
3508    NCZ05_CHUKWUDI019
3509    NCZ0

In [32]:
y = df_test['Product_Price']

In [33]:
subm7 = pd.DataFrame({'Product_Supermarket_Identifier':df_test.Product_Supermarket_Identifier,
              'Product_Supermarket_Sales': y}).set_index('Product_Supermarket_Identifier')
subm7.to_csv('subm7.csv')

In [19]:
df = pd.read_csv('nsubmission.csv')
y = df.Product_Supermarket_Sales
y.head(8)

0     911.798191
1    7198.708485
2    7392.048313
3     919.679251
4    7326.104520
5    4317.247975
6    7820.318056
7    4022.280033
Name: Product_Supermarket_Sales, dtype: float64

In [34]:
df_7 = pd.read_csv('subm7.csv')
y7 = df_7.Product_Supermarket_Sales

In [36]:
df_9 = pd.read_csv('subm9.csv')
y9 = df_9.Product_Supermarket_Sales

In [37]:
# Ensemble Averaging to create a second choice submission to kaggle
df_test['best'] = (y7+y9)/2

In [43]:
df_test

,Product_Identifier,Supermarket_Identifier,Product_Supermarket_Identifier,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type,Pred,best
0,DRA59,CHUKWUDI010,DRA59_CHUKWUDI010,8.270,Normal Fat,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store,1065.368418,1001.811831
1,DRA59,CHUKWUDI013,DRA59_CHUKWUDI013,8.270,Normal Fat,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1,7285.873916,7537.625015
2,DRB01,CHUKWUDI013,DRB01_CHUKWUDI013,7.390,Low Fat,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1,7480.172244,7738.636994
3,DRB13,CHUKWUDI010,DRB13_CHUKWUDI010,6.115,Normal Fat,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store,1094.667323,1029.362854
4,DRB13,CHUKWUDI013,DRB13_CHUKWUDI013,6.115,Normal Fat,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1,7413.578059,7669.741759
5,DRB25,CHUKWUDI017,DRB25_CHUKWUDI017,12.300,Low Fat,0.069853,Soft Drinks,268.73,2014,NaN,Cluster 2,Supermarket Type1,4525.142858,4356.286228
6,DRB25,CHUKWUDI027,DRB25_CHUKWUDI027,NaN,Low Fat,0.069123,Soft Drinks,265.23,1992,Medium,Cluster 3,Supermarket Type3,4311.546787,6955.954736
7,DRB25,CHUKWUDI035,DRB25_CHUKWUDI035,12.300,Low Fat,0.069447,Soft Drinks,265.98,2011,Small,Cluster 2,Supermarket Type1,4541.459551,4311.706958
8,DRB48,CHUKWUDI017,DRB48_CHUKWUDI017,16.750,Normal Fat,0.024994,Soft Drinks,93.71,2014,NaN,Cluster 2,Supermarket Type1,1577.982128,1519.099402
9,DRB48,CHUKWUDI027,DRB48_CHUKWUDI027,NaN,Normal Fat,0.024733,Soft Drinks,100.71,1992,Medium,Cluster 3,Supermarket Type3,1637.129574,2641.232898


In [24]:
col = pd.DataFrame({'y_8': y, "ypred":df_test.Product_Price})
col.head()

,y_8,ypred
0,911.798191,1083.663662
1,7198.708485,7635.570029
2,7392.048313,7839.193988
3,919.679251,1113.465709
4,7326.104520,7769.403518


In [47]:
col['new'] = col.min(axis=1)
col['Supermarket_Type'] = df_test.Supermarket_Type
col.head()

,y_8,ypred,new,Supermarket_Type
0,919.96,1001.811831,919.96,Grocery Store
1,7439.68,7537.625015,7439.68,Supermarket Type1
2,7638.08,7738.636994,7638.08,Supermarket Type1
3,945.26,1029.362854,945.26,Grocery Store
4,7570.08,7669.741759,7570.08,Supermarket Type1


In [55]:
d = col.where(col['Supermarket_Type'] == 'Supermarket Type3').loc[:, "y_8", "ypred"]
d.new = d.max(axis=1)# and d.new != None

IndexingError: Too many indexers

In [62]:
col.head(10)

,y_8,ypred,new,Supermarket_Type
0,919.96,1001.811831,919.96,Grocery Store
1,7439.68,7537.625015,7439.68,Supermarket Type1
2,7638.08,7738.636994,7638.08,Supermarket Type1
3,945.26,1029.362854,945.26,Grocery Store
4,7570.08,7669.741759,7570.08,Supermarket Type1
5,4299.68,4356.286228,4299.68,Supermarket Type1
6,6895.98,6955.954736,6895.98,Supermarket Type3
7,4255.68,4311.706958,4255.68,Supermarket Type1
8,1499.36,1519.099402,1499.36,Supermarket Type1
9,2618.46,2641.232898,2618.46,Supermarket Type3


In [40]:
subm11 = pd.DataFrame({'Product_Supermarket_Identifier':df_test.Product_Supermarket_Identifier,
              'Product_Supermarket_Sales': df_test['best']}).set_index('Product_Supermarket_Identifier')
subm11.to_csv('subm11.csv')